In [7]:
from pycocotools.coco import COCO
from PIL import Image, ImageDraw
from pycocotools import mask as maskUtils
import os
import numpy as np

In [ ]:
def create_masks_from_coco_subset(image_dir, coco_json_path, output_mask_dir):
    """
    Tạo mask từ file COCO JSON và lưu vào thư mục, với tên file mask giống tên file ảnh gốc.
    Chỉ tạo mask cho những ảnh có thông tin trong file annotation.

    Args:
        image_dir: Đường dẫn đến thư mục chứa ảnh gốc.
        coco_json_path: Đường dẫn đến file COCO JSON.
        output_mask_dir: Đường dẫn đến thư mục để lưu mask.
    """

    os.makedirs(output_mask_dir, exist_ok=True)

    try:
        coco = COCO(coco_json_path)
    except FileNotFoundError:
        print(f"Error: COCO JSON file not found at {coco_json_path}")
        return
    except Exception as e:
        print(f"Error loading COCO JSON: {e}")
        return

    # Lặp qua các ảnh có trong file annotation (thay vì tất cả các ảnh có trong thư mục)
    for img_id in coco.imgs:
        img_data = coco.imgs[img_id]
        img_width = img_data['width']
        img_height = img_data['height']
        file_name = img_data['file_name']

        # Kiểm tra xem ảnh có tồn tại trong thư mục ảnh không (tùy chọn)
        image_path = os.path.join(image_dir, file_name)
        if not os.path.exists(image_path):
            print(f"Warning: Image file {file_name} not found in {image_dir}. Skipping.")
            continue  # Bỏ qua ảnh này nếu không tìm thấy

        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)

        mask = Image.new('L', (img_width, img_height), 0)
        draw = ImageDraw.Draw(mask)

        for ann in anns:
            if 'segmentation' in ann and isinstance(ann['segmentation'], list):  # Polygon
                for seg in ann['segmentation']:
                    poly = [(seg[i], seg[i + 1]) for i in range(0, len(seg), 2)]
                    draw.polygon(poly, outline=255, fill=255)
            elif 'segmentation' in ann and isinstance(ann['segmentation'], dict) and 'counts' in ann['segmentation']: # RLE
                rle = ann['segmentation']
                mask_rle = maskUtils.decode(rle)
                mask_pil = Image.fromarray(mask_rle * 255)
                mask.paste(mask_pil, (0, 0), mask_pil)

        mask_path = os.path.join(output_mask_dir, file_name)
        if not mask_path.lower().endswith('.png'):
            mask_path = os.path.splitext(mask_path)[0] + '.png'
        try:
            mask.save(mask_path)
            print(f"Đã tạo và lưu mask: {mask_path}")
        except Exception as e:
            print(f"Error saving mask: {e}")


In [27]:
image_dir = './dataset/train/images'  # Thay bằng đường dẫn đến thư mục chứa ảnh
coco_json_path = './annotations/Pawn/pawn_train_coco.json'  # Thay bằng đường dẫn đến file COCO JSON
output_mask_dir = './dataset/train/masks'  # Thay bằng thư mục bạn muốn lưu mask

In [28]:
create_masks_from_coco_subset(image_dir, coco_json_path, output_mask_dir)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_1.png
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_100.png
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_102.png
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_103.png
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_105.png
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_106.png
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_107.png
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_108.png
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_111.png
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_112.png
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_113.png
Đã tạo và lưu mask: ./dataset/train/masks\brightness_normalized_pawn_114